In [1]:
import sys
from pathlib import Path
from types import SimpleNamespace

import numpy as np
import pandas as pd
import torch
from torchvision.transforms import (
    Resize,
    RandomCrop,
    CenterCrop,
    RandomHorizontalFlip,
    RandomRotation,
    ColorJitter,
    Normalize,
    Compose,
)

PROJECT_ROOT = Path(sys.path[0]).parent.resolve()
sys.path.append(PROJECT_ROOT)

from src.utils import load_data
from src.modules import ResNetRankNet
from src.datasets import FlowPhotoDataset

/tmp/ipykernel_3803923/227242812.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/opt/conda/envs/fpe-model.dev-amrita/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = SimpleNamespace(
    inference_data_file="/home/amritagupta/ssdprivate/data/Streamflow/fpe_stations/West Brook Lower_01171090/FLOW_CFS/images.csv",
    inference_image_root_dir="/home/amritagupta/ssdprivate/data/Streamflow/fpe_stations/West Brook Lower_01171090/FLOW_CFS",
    col_label='value',
    img_sample_mean=[0.35581599, 0.34419223, 0.2620608],
    img_sample_std=[0.22301329, 0.22242821, 0.22193837],
    augment=True,
    normalize=True,
    batch_size=64,
    gpu=0,
    ckpt_path = '../../../edge-device-fpe-model-test/model.pth'
)

In [3]:
print(Path.exists(Path(args.ckpt_path)))

True


In [4]:
def create_image_transforms(
    resize_shape,
    input_shape,
    augmentation=True,
    normalization=True,
    means=None,
    stds=None,
):
    image_transforms = {
        "train": [
            Resize(resize_shape),
        ],
        "eval": [
            Resize(resize_shape),
        ],
    }

    # augmentation
    image_transforms["train"].extend(
        [
            RandomCrop(input_shape),
            RandomHorizontalFlip(),
            RandomRotation(10),
            ColorJitter(),
        ]  # type: ignore
    ) if augmentation else image_transforms["train"].append(
        CenterCrop(input_shape)  # type: ignore
    )  # type: ignore
    image_transforms["eval"].append(CenterCrop(input_shape))  # type: ignore

    # normalization
    if normalization:
        image_transforms["train"].append(Normalize(means, stds))  # type: ignore
        image_transforms["eval"].append(Normalize(means, stds))  # type: ignore

    # composition
    image_transforms["train"] = Compose(image_transforms["train"])  # type: ignore
    image_transforms["eval"] = Compose(image_transforms["eval"])  # type: ignore
    return image_transforms

In [5]:
df = load_data(args.inference_data_file)
ds = FlowPhotoDataset(df, args.inference_image_root_dir, col_label=args.col_label)
image = ds.get_image(0)
aspect = image.shape[2] / image.shape[1]
# set up image transforms
resize_shape = [480, np.int32(480 * aspect)]
input_shape = [384, np.int32(384 * aspect)]
image_transforms = create_image_transforms(
    resize_shape,
    input_shape,
    means=args.img_sample_mean,
    stds=args.img_sample_std,
    augmentation=args.augment,
    normalization=args.normalize,
)
ds.transform = image_transforms["eval"]  # use eval transforms during inference
dl = torch.utils.data.DataLoader(
    ds, batch_size=args.batch_size, shuffle=False, num_workers=24
)

# # # # # # # # # # # # # # # # # # # # # # # # #
# LOAD TRAINED MODEL
# # # # # # # # # # # # # # # # # # # # # # # # #
if torch.cuda.is_available():
    device = torch.device(f"cuda:{args.gpu}")
    print(f"Try to put data and model on GPU {args.gpu} for inference.")
else:
    device = torch.device("cpu")
    print("Try to put data and model on CPU for inference.")
model = ResNetRankNet(
    input_shape=(3, input_shape[0], input_shape[1]),
    resnet_size=18,
    truncate=2,
    pretrained=True,
)
model = torch.nn.DataParallel(
    model,
    # device_ids=[
    #     args.gpu,
    # ],
)
# model.to(device)
checkpoint = torch.load(args.ckpt_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()
print("Loaded model from checkpoint.")


Try to put data and model on GPU 0 for inference.
Loaded model from checkpoint.


In [6]:
# check if model is on GPU or CPU
if next(model.parameters()).is_cuda:
    print("Model is on GPU.")
else:
    print("Model is on CPU.")

Model is on CPU.


In [7]:
model.to(device)

# check now if model is on GPU or CPU
if next(model.parameters()).is_cuda:
    print("Model is on GPU.")
else:
    print("Model is on CPU.")

Model is on GPU.


In [8]:
from tqdm import tqdm

In [9]:
scores = np.empty((len(dl.dataset),))
sidx = 0
with torch.no_grad():
    for bidx, batch in tqdm(enumerate(dl), total=len(dl)):
        inputs, labels = batch
        nsamples = labels.shape[0]
        outputs = model.module.forward_single(inputs.to(device))
        scores[sidx : sidx + nsamples] = outputs.detach().cpu().numpy()
        sidx += nsamples
dl.dataset.table.loc[:, "scores"] = scores
print("Inference complete.")

100%|██████████| 1597/1597 [06:42<00:00,  3.97it/s]

Inference complete.
